<a href="https://colab.research.google.com/github/CoderJeel/Fruits-recognition/blob/master/Model%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Fri Dec 11 04:38:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python3 -c 'import tensorflow as tf; print(tf.__version__)'  # for Python 3


2020-10-30 06:45:10.297708: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2.3.0


#Upload your kaggle token

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json 

In [ ]:
!kaggle datasets download -d chrisfilo/fruit-recognition

100% 7.91G/7.92G [02:57<00:00, 42.8MB/s]
100% 7.92G/7.92G [02:57<00:00, 47.9MB/s]


In [ ]:
!unzip /content/fruit-recognition.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: data/Pomegranate/Pomegranet00323.png  
  inflating: data/Pomegranate/Pomegranet00324.png  
  inflating: data/Pomegranate/Pomegranet00325.png  
  inflating: data/Pomegranate/Pomegranet00326.png  
  inflating: data/Pomegranate/Pomegranet00327.png  
  inflating: data/Pomegranate/Pomegranet00328.png  
  inflating: data/Pomegranate/Pomegranet00329.png  
  inflating: data/Pomegranate/Pomegranet0033.png  
  inflating: data/Pomegranate/Pomegranet00330.png  
  inflating: data/Pomegranate/Pomegranet00331.png  
  inflating: data/Pomegranate/Pomegranet00332.png  
  inflating: data/Pomegranate/Pomegranet00333.png  
  inflating: data/Pomegranate/Pomegranet00334.png  
  inflating: data/Pomegranate/Pomegranet00335.png  
  inflating: data/Pomegranate/Pomegranet00336.png  
  inflating: data/Pomegranate/Pomegranet00337.png  
  inflating: data/Pomegranate/Pomegranet00338.png  
  inflating: data/Pomegranate/Pomegranet00339.png  
  inflating: d

In [ ]:
import os
main_dir = os.listdir('/content/data/')

for i in main_dir:
  full_path = os.path.join('/content/data',i)
  print('categories:',i,'number of files',len(os.listdir(full_path)))

categories: Banana number of files 3027
categories: Orange number of files 3012
categories: Apple number of files 7
categories: Pitaya number of files 2501
categories: Tomatoes number of files 2171
categories: Pomegranate number of files 2167
categories: Persimmon number of files 2072
categories: Carambola number of files 2080
categories: Mango number of files 4154
categories: Guava number of files 4
categories: Peach number of files 2629
categories: Pear number of files 3012
categories: Kiwi number of files 4
categories: Plum number of files 2298
categories: muskmelon number of files 2078


In [ ]:
%ls '/content/data/Apple/Total Number of Apples' | wc -l
%ls '/content/data/Guava/Guava total' | wc -l
%ls '/content/data/Kiwi/Total Number of Kiwi fruit' | wc -l

5024
12552
4173


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Dense,MaxPool2D,Conv2D,Activation,Dropout,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=90,
        horizontal_flip=True,
        validation_split=0.05)
train_generator = train_datagen.flow_from_directory(
        '/content/data',
        target_size=(80,65),
        batch_size=128,color_mode='grayscale',
        class_mode='categorical',
        
        subset='training')

val_generator = train_datagen.flow_from_directory(
        '/content/data',
        target_size=(80,65),
        batch_size=128,color_mode='grayscale',
        class_mode='categorical',
        subset='validation')


Found 67029 images belonging to 15 classes.
Found 3520 images belonging to 15 classes.
Found 53 images belonging to 15 classes.


In [ ]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=089680e9ec71e54b0c3a51224317c3eef4409dafb56295b85b7b9140d1f3fa02
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=8e834d33438317c838cc2168cede8c97d963c80a43a6b590cbbd4f77026a807c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import datetime
import os
from tensorflow import keras
import tensorflow as tf
from sklearn.metrics import f1_score
#from kerastuner.tuners import RandomSearch
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
def build_model(hp):
  model = keras.Sequential()
  model.add(Conv2D(64,(3,3),input_shape=(80,65,1),activation='relu'))
  for i in range(hp.Int('num_layers', 1, 3)):
    model.add(Conv2D(hp.Int(str(i),min_value=32,max_value=128,step=32),(3,3),activation='relu'))
  model.add(MaxPool2D(pool_size=(3,3)))
  model.add(Flatten())
  for i in range(hp.Int('num_layers', 1, 5)):
    model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=128,
                                            step=32),
                               activation='relu'))
  model.add(Dense(15,activation='softmax'))
  model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['Precision'])
  return model

"def build_model(hp):\n  model = keras.Sequential()\n  model.add(Conv2D(64,(3,3),input_shape=(80,65,1),activation='relu'))\n  for i in range(hp.Int('num_layers', 1, 3)):\n    model.add(Conv2D(hp.Int(str(i),min_value=32,max_value=128,step=32),(3,3),activation='relu'))\n  model.add(MaxPool2D(pool_size=(3,3)))\n  model.add(Flatten())\n  for i in range(hp.Int('num_layers', 1, 5)):\n    model.add(Dense(units=hp.Int('units_' + str(i),\n                                            min_value=32,\n                                            max_value=128,\n                                            step=32),\n                               activation='relu'))\n  model.add(Dense(15,activation='softmax'))\n  model.compile(\n        optimizer=tf.keras.optimizers.Adam(\n            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),\n        loss='categorical_crossentropy',\n        metrics=['Precision'])\n  return model"

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_loss', 
    max_trials=5,
    executions_per_trial=2,
    directory='my_dir',
    project_name='fr_clsf')


"tuner = RandomSearch(\n    build_model,\n    objective='val_loss', \n    max_trials=5,\n    executions_per_trial=2,\n    directory='my_dir',\n    project_name='fr_clsf')"

In [ ]:
tuner.search(train_generator,
             epochs=5,
             validation_data=val_generator,callbacks=[tensorboard_callback],steps_per_epoch=50)

'tuner.search(train_generator,\n             epochs=5,\n             validation_data=val_generator,callbacks=[tensorboard_callback],steps_per_epoch=50)'

#Model Define 

In [ ]:

model = Sequential([
                    Conv2D(128,(3,3),input_shape=(80,65,1),activation='relu'),
                    Conv2D(128,(3,3),activation='relu'),
                    Conv2D(128,(3,3),activation='relu'),
                    MaxPool2D(pool_size=(3,3)),
                    Flatten(),
                    Dense(128,activation='relu'),
                    Dense(128,activation='relu'),
                   Dense(128,activation='relu'),
                   Dense(128,activation='relu'),
                    Dense(15,activation='softmax')])
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=[tf.keras.metrics.Recall()])

"model = Sequential([\n                    Conv2D(128,(3,3),input_shape=(80,65,1),activation='relu'),\n                    Conv2D(128,(3,3),activation='relu'),\n                    Conv2D(128,(3,3),activation='relu'),\n                    MaxPool2D(pool_size=(3,3)),\n                    Flatten(),\n                    Dense(128,activation='relu'),\n                    Dense(128,activation='relu'),\n                   Dense(128,activation='relu'),\n                   Dense(128,activation='relu'),\n                    Dense(15,activation='softmax')])\nmodel.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=[tf.keras.metrics.Recall()])"

In [ ]:
model.summary()

#Model training

In [ ]:
model.fit(train_generator,validation_data=val_generator,epochs=10,callbacks=[tensorboard_callback])

Epoch 1/10
  1/524 [..............................] - ETA: 0s - loss: 0.0494 - recall: 0.9688WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
524/524 [==============================] - 344s 657ms/step - loss: 0.0974 - recall: 0.9641 - val_loss: 0.2136 - val_recall: 0.9253
Epoch 2/10
524/524 [==============================] - 341s 650ms/step - loss: 0.0961 - recall: 0.9645 - val_loss: 0.3923 - val_recall: 0.8753
Epoch 3/10
524/524 [==============================] - 341s 651ms/step - loss: 0.0937 - recall: 0.9651 - val_loss: 0.2625 - val_recall: 0.9156
Epoch 4/10
524/524 [==============================] - 337s 644ms/step - loss: 0.0993 - recall: 0.9630 - val_loss: 0.1842 - val_recall: 0.9366
Epoch 5/10
524/524 [==============================] - 338s 645ms/step - 

In [ ]:
model.save('/content/drive/My Drive/Projects/Fruites Classifier/Models/model_v15.h5')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
model.evaluate(test_generator)

1/1 [==============================] - 0s 2ms/step - loss: 7.2836 - recall: 0.2075


[7.283633232116699, 0.2075471729040146]